# Практическое занятие №2 - локализация объектов на изображениях

1. Разметка данных с помощью виджета QSL для Jupyter (https://github.com/faustomorales/qsl)
2. Знакомство с архитектурой нейронных сетей для локализации объектов на изображении.
3. Разработка простой системы управления по данным с видео-камеры.

In [1]:
!pip install -U -q qsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.4 MB/s eta 0:00:00


In [2]:
# These next two lines are only required on Google Colab.
# They are not needed in Jupyter Lab or Jupyter Notebook.
import google.colab
google.colab.output.enable_custom_widget_manager()

In [3]:
# Widgets have been tested on JupyterLab >= 3 and
# Jupyter Notebook >= 6
import qsl
import qsl.testing.files

In [5]:
qsl.testing.files.TEST_IMAGES[0]

'/usr/local/lib/python3.8/dist-packages/qsl/testing/data/image1.jpg'

In [69]:
%%writefile images.csv
target
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_002.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_003.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_005.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_006.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_007.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_008.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_009.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_010.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_3/image_011.jpg

Overwriting images.csv


In [70]:
import pandas

image_file = pandas.read_csv('images.csv')

In [71]:
labeler = qsl.MediaLabeler(
    items=image_file.to_dict(orient='records'),
    config={
        "regions": [
            {"name": "Object", "multiple": True, "options": [{"name": "bottle"}, {"name": "bug"}]}
        ]
    })
labeler.mode = "dark"
labeler

MediaLabeler(buttons={'prev': False, 'next': True, 'save': True, 'config': True, 'delete': False, 'ignore': Tr…

In [80]:
# You can access the current state of the image labels at any time.
# Mostly self-explanatory. The segmentation maps provide map
# dimensions (width/height) and a run-length encoded representation
# with classes i (inside mask) and o (outside of mask). The mask
# may be lower-res than the image (you can tell from the dimensions)
# if the image was large, so you'll have to upsample and interpolate
# the map up to true image size. This little twist makes the in-browser
# behavior more memory efficient at the cost of mask precision.
with open('result.json', 'w') as file:
    file.write(str(labeler.items).replace("'", '"'))

In [82]:
labels = None
with open('result.json', 'r') as file:
    labels = file.read()
    print(labels)

[{"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg", "labels": {"image": {}, "polygons": [{"points": [{"x": 0.642587170887295, "y": 0.3516193884040677}, {"x": 0.6700005413571787, "y": 0.3926353214413227}, {"x": 0.7643025357735784, "y": 0.45122951149454404}, {"x": 0.8235154159885271, "y": 0.5117768412162061}, {"x": 0.8728594828343176, "y": 0.5723241709378682}, {"x": 0.9046589925793826, "y": 0.5977149866275975}, {"x": 0.909045131854564, "y": 0.6191995229804453}, {"x": 0.934265432686857, "y": 0.6133401039751232}, {"x": 0.9638718727943313, "y": 0.6367777799964117}, {"x": 0.9265896889552896, "y": 0.7012313890549553}, {"x": 0.8893075051162478, "y": 0.7656849981134988}, {"x": 0.8706664131967269, "y": 0.8008415121454316}, {"x": 0.8114535329817782, "y": 0.8027946518138723}, {"x": 0.7686886750487598, "y": 0.785216394797906}, {"x": 0.7423718393976715, "y": 0.7754506964557024}, {"x": 0.7335995608473087, "y": 0.7344347634184474}, {"x": 0.725

In [85]:
import json

json.loads(labels)

JSONDecodeError: ignored

In [49]:
import qsl

params = dict(
    config={
        "image": [
            {"name": "Location", "multiple": False, "options": [{"name": "Indoor"}, {"name": "Outdoor"}]},
            {"name": "Flags", "multiple": True, "freeform": True},
            {"name": "Type", "multiple": False, "options": [{"name": "Cat"}, {"name": "Dog"}]},
        ],
        "regions": [
            {"name": "Type", "multiple": False, "options": [{"name": "Eye"}, {"name": "Nose"}]}
        ]
    },
    items=[
        {"target": "https://picsum.photos/id/1025/500/500", "defaults": {"image": {"Type": ["Dog"]}}},
    ],
)
qsl.MediaLabeler(**params)

MediaLabeler(buttons={'prev': False, 'next': False, 'save': True, 'config': True, 'delete': False, 'ignore': T…